# Cypher queries

## Testing import from blocks and transaction topics by sample queries

In [2]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "neo4j"))

## Example Data

In [3]:
# block node
b1 = {
        "block_hash": "00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395ba2",
        "block_height": "681545",
        "block_timestamp" : "2021-05-02 16:50:00",
        "p_block_hash": "00"
    }

b2 = {
        "block_hash": "00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395986",
        "block_height": "6815765",
        "block_timestamp" : "2021-05-02 17:00:00",
        "p_block_hash": "00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395ba2"
    }

b3 = {
        "block_hash": "00000000000000000008bb27dd339ad1238912s73981273981d7298127491824",
        "block_height": "6815766",
        "block_timestamp" : "2021-05-02 17:00:00",
        "p_block_hash": "00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395986"
    }

# transaction node 
t0 = {
    "txid": "0",
    "block_date": "2021-05-02",
    "inDegree": 1,
    "outDegree": 2, 
    "inSum": 10,
    "outSum": 10
}

t1 = {
    "txid": "cebc8ac65d27d9ae05aad5249e5b991e01d50b256118ea1722373a212814b81e",
    "block_date": "2021-05-02",
    "inDegree": 1,
    "outDegree": 4, 
    "inSum": 20.0,
    "outSum": 13.07
}

# address node 
a1 = {
    "address": "1EepjXgvWUoRyNvuLSAxjiqZ1QqKGDANLW",
    "inDegree": 1, 
    "outDegree": 1
}

# FOREACH Test
f0 = {
    "txid" : "0",
    "addresses": [
        {
            "addr": "id_addr0",
            "output_nr": 0
        },
        {
            "addr": "id_addr1",
            "output_nr": 1
        },
        
    ], 
    "block_hash":'00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395ba2'
    
}

f1 = {
    "txid" : "1",
    "addresses": [
        {
            "addr": "id_addr1_0",
            "output_nr": 0
        },
        {
            "addr": "id_addr1_1",
            "output_nr": 1
        },
        
    ],
    "input_txid":"0",
    "input_vout":1,
    "block_hash":'00000000000000000008bb27dd339ad1238912s73981273981d7298127491824'
    
}


In [65]:
%run bitcoin\ json-rpc-functions.ipynb

In [66]:
height = 600000
block_hash = getBlockHash(height)
print('Block Nr:', height, '- Hash: ', block_hash)
data, block = getblock(block_hash)#
display(data)
#block

Block Nr: 600000 - Hash:  00000000000000000007316856900e76b4f7a9139cfbfba89842c8d196cd5f91


{'block_hash': '00000000000000000007316856900e76b4f7a9139cfbfba89842c8d196cd5f91',
 'block_height': 600000,
 'block_median_time': '2019-10-19T02:04',
 'block_date': '2019-10-19',
 'previousblockhash': '00000000000000000003ecd827f336c6971f6f77a0b9fba362398dd867975645'}

In [67]:
transactions = block['tx']
tx = gettx(transactions[1])
tx

{'txid': 'a8178a7223372414ac060b4bba4b33b8b4847a756fa76a715af7fd11bfd143d5',
 'block_hash': '00000000000000000007316856900e76b4f7a9139cfbfba89842c8d196cd5f91',
 'block_date': '2019-10-19',
 'outDegree': 3,
 'inDegree': 1,
 'outSum': 138819784059,
 'inSum': 138819884059,
 'input_list': [{'addr': '17A16QmavnUfCW11DAApiJxp7ARnxN5pGX',
   'val': 138819884059}],
 'output_list': [{'addr': '3QKAn2B1uDquujLZnoynVoq1M9uac66Ysr',
   'outNr': 0,
   'val': 0.00795759},
  {'addr': '1F8fDpYbMLMaz1tBEehqPJSN8XTL6t5TDz',
   'outNr': 1,
   'val': 0.01241006},
  {'addr': '17A16QmavnUfCW11DAApiJxp7ARnxN5pGX',
   'outNr': 2,
   'val': 1388.17747294}]}

## Transactions


In [ ]:
def create_transaction(tx, txid, block_date, inDegree, outDegree, inSum, outSum):
    tx.run('MERGE (t:Transaction{txid: $txid, block_date: $block_date, inDegree: $inDegree, outDegree: $outDegree, inSum: $inSum, outSum: $outSum})',
           txid=txid, block_date=block_date, inDegree=inDegree, outDegree=outDegree, inSum=inSum, outSum=outSum)

In [ ]:
t = t1
with driver.session() as session:
    session.write_transaction(create_transaction, 
                              t['txid'],
                              t['block_date'],
                              t['inDegree'],
                              t['outDegree'],
                              t['inSum'],
                              t['outSum'])

#### FOREACH Test

In [ ]:
def test_foreach(tx, txid, addresses):
    tx.run('MERGE (f:Transaction{txid: $txid}) '
           'FOREACH (address in $addresses | '
           ' MERGE (a:Address{txid: address.addr}) '
           ' MERGE (f)-[r:RECEIVES{output_nr: address.output_nr}]->(a))',
           txid=txid, addresses=addresses)

In [ ]:
query = '''
    MATCH (input_t:Transaction)-[input_r:RECEIVES]->(input_a:Address) 
        WHERE 
            input_t.txid ="0" AND 
            input_r.output_nr = 0 
            
    
    MERGE (t:Transaction{txid: $txid}) 
    
    MERGE (input_a)-[:Sends]->(t)
    MERGE (b:Block{hash: $blockhash})
    MERGE (t)-[:BELONGS_TO]->(b)
    
    FOREACH (address in $addresses | 
        MERGE (a:Address{txid: address.addr}) 
        MERGE (t)-[r:RECEIVES{output_nr: address.output_nr}]->(a))
    '''

def test_foreach(tx, txid, addresses, blockhash):
    tx.run(query, txid=txid, addresses=addresses, blockhash=blockhash)

In [ ]:
f = f1
with driver.session() as session:
    session.write_transaction(test_foreach, f["txid"], f["addresses"], f["block_hash"])

#### Next step testing 13.05.

In [51]:
### Current Structure
t0 = {
    "txid": "0000000fdb4d9b9479d9b061921cb9d073f1c814a3bbef30ccae212ab651952d",
    "block_hash": "000000000000000000069eef0a89044c11d96e2f9c33b4c5a8f7e402c717cb10",
    "address_list": [
        {
            "addr": "1Nvv2vdKssYTJNPYkqmsHHf9Kn7ei4nbXu",
            "output_nr": 0
        },
        {
            "addr": "1gucZbN2LdKr2z3szVWx1ZiQNeYgXK6py",
            "output_nr": 1
        }
    ],
    "inDegree": 2,
    "outDegree": 2,
    "outSum": 2431925,
    "input_list": [
        {
            "txid": "11f3cdf4ed881811d5cf710e7fb8e6115768c999990a35b8f092e5e94adfbd5d",
            "output_nr": 0
        },
        {
            "txid": "e365bf7ee8f4466096743d53b327cddc2054df284f46907abc9a84477f91470d",
            "output_nr": 1
        }
    ]
}


### Current Structure + date + out.value
t1 = {
    "txid": "0000000fdb4d9b9479d9b061921cb9d073f1c814a3bbef30ccae212ab651952d",
    "block_hash": "000000000000000000069eef0a89044c11d96e2f9c33b4c5a8f7e402c717cb10",
    "address_list": [
        {
            "addr": "1Nvv2vdKssYTJNPYkqmsHHf9Kn7ei4nbXu",
            "output_nr": 0,
            "value": 23000
        },
        {
            "addr": "1gucZbN2LdKr2z3szVWx1ZiQNeYgXK6py",
            "output_nr": 1,
            "value": 1 
        }
    ],
    "inDegree": 2,
    "outDegree": 2,
    "outSum": 2431925,
    "date":"2020-03_30",
    "input_list": [
        {
            "txid": "11f3cdf4ed881811d5cf710e7fb8e6115768c999990a35b8f092e5e94adfbd5d",
            "output_nr": 0

        },
        {
            "txid": "e365bf7ee8f4466096743d53b327cddc2054df284f46907abc9a84477f91470d",
            "output_nr": 1
        }
    ]
}

### Current Structure + date + out.value
t2 = {
    "txid": "0000000fdb4d9b9479d9b061921cb9d073f1c814a3bbef30ccae212ab651952d",
    "block_hash": "000000000000000000069eef0a89044c11d96e2f9c33b4c5a8f7e402c717cb10",
    "address_list": [
        {
            "addr": "1Nvv2vdKssYTJNPYkqmsHHf9Kn7ei4nbXu",
            "output_nr": 0,
            "value": 23000
        },
        {
            "addr": "1gucZbN2LdKr2z3szVWx1ZiQNeYgXK6py",
            "output_nr": 1,
            "value": 1 
        }
    ],
    "inDegree": 2,
    "outDegree": 2,
    "outSum": 2431925,
    "input_list": [
        {
            "addr": "11f3cdf4ed881811d5cf710e7fb8e6115768c999990a35b8f092e5e94adfbd5d",
            "output_nr": 0
        },
        {
            "txid": "e365bf7ee8f4466096743d53b327cddc2054df284f46907abc9a84477f91470d",
            "output_nr": 1
        }
    ]
}




##### NOTES & fragments

'''
    MATCH (input_t:Transaction)-[input_r:RECEIVES]->(input_a:Address) 
        WHERE 
            input_t.txid ="0" AND 
            input_r.output_nr = 0 
    MERGE (input_a)-[:Sends]->(t)
'''

'''
    MERGE (t:Transaction{txid: $txid}) 
        SET t = {txid: $txid, inDegree: $inDegree, outDegree: $outDegree, outSum: $outSum}

    MERGE (b:Block{hash: $block_hash})
    MERGE (t)-[:BELONGS_TO]->(b)
    
    FOREACH (o_address in $address_list | 
        MERGE (o_a:Address{txid: o_address.addr}) 
        MERGE (t)-[r:RECEIVES{output_nr: o_address.output_nr}]->(o_a))
        
    FOREACH (input in $input_list | 
        MATCH (i_tx:Transaction)-[i_r:RECEIVES]->(i_a:Address)
        WHERE
            i_tx.txid = input.txid AND 
            i_r.output_nr = input.output_nr 
        MERGE (i_a)-[:Sends]->(t))
    '''

In [57]:
###Current Structure

## Amend
# - input receives value
# - input sends value


query = '''
    MERGE (t:Transaction{txid: $txid}) 
        SET t = {txid: $txid, inDegree: $inDegree, outDegree: $outDegree, outSum: $outSum}

    MERGE (b:Block{hash: $block_hash})
    MERGE (t)-[:BELONGS_TO]->(b)
    
    FOREACH (o_address in $address_list | 
        MERGE (o_a:Address{address: o_address.addr}) 
            ON CREATE SET o_a = {address: o_address.addr, inDegree: 1}
            ON MATCH  SET o_a = {inDegree: o_a.inDegree + 1}
        MERGE (t)-[r:RECEIVES{output_nr: o_address.outNr}]->(o_a))
        
    FOREACH (input in $input_list | 
        MERGE (i_tx:Transaction{txid: input.txid})-[i_r:RECEIVES{output_nr: input.output_nr, value: 2}]->(i_a:Address)
            ON MATCH SET i_a = {outDegree: i_a.outDegree + 1}
        MERGE (i_a)-[:Sends{value: i_r.value}]->(t))
    '''

def run_tx(tx, event):
    tx.run(query, 
           txid         = event["txid"], 
           address_list = event["address_list"], 
           block_hash   = event["block_hash"], 
           inDegree     = event["inDegree"], 
           outDegree    = event["outDegree"], 
           outSum       = event["outSum"],
           input_list   = event["input_list"])
    
t = t0

In [58]:
###Current Structure + amendmends

query = '''
    MERGE (t:Transaction{txid: $txid}) 
        SET t = {txid: $txid, inDegree: $inDegree, outDegree: $outDegree, outSum: $outSum, date: $date}

    MERGE (b:Block{hash: $block_hash})
    MERGE (t)-[:BELONGS_TO]->(b)
    
    FOREACH (o_address in $address_list | 
        MERGE (o_a:Address{address: o_address.addr}) 
            ON CREATE SET o_a = {address: o_address.addr, inDegree: 1}
            ON MATCH  SET o_a = {inDegree: o_a.inDegree + 1}
        MERGE (t)-[r:RECEIVES{output_nr: o_address.output_nr, value: o_address.value}]->(o_a))
        
    FOREACH (input in $input_list | 
        MERGE (i_tx:Transaction{txid: input.txid})-[i_r:RECEIVES{output_nr: input.output_nr, value: 2}]->(i_a:Address)
            ON MATCH SET i_a = {outDegree: i_a.outDegree + 1}
        MERGE (i_a)-[:Sends{value: i_r.value}]->(t))
        
    '''
# increment the outdegree

def run_tx(tx, event):
    tx.run(query, 
           txid         = event["txid"], 
           address_list = event["address_list"], 
           block_hash   = event["block_hash"], 
           inDegree     = event["inDegree"], 
           outDegree    = event["outDegree"], 
           outSum       = event["outSum"],
           input_list   = event["input_list"], 
           date         = event["date"])
t = t1

In [80]:
###Address matching Structure + amendmends

query = '''
    MERGE (t:Transaction{txid: $txid}) 
        SET t += {inDegree: $inDegree, outDegree: $outDegree, outSum: $outSum, date: $date}

    MERGE (b:Block{hash: $block_hash})
    MERGE (t)-[:BELONGS_TO]->(b)
    
    FOREACH (output in $output_list | 
        MERGE (o_a:Address{address: output.addr}) 
            ON CREATE SET o_a += {inDegree: 1, outDegree: 0}
            ON MATCH  SET o_a += {inDegree: o_a.inDegree + 1}
        MERGE (t)-[r:RECEIVES{output_nr: output.outNr, value: output.val}]->(o_a))
        
    FOREACH (input in $input_list | 
        MERGE (i_a:Address{address: input.addr})
            ON MATCH SET i_a += {outDegree: i_a.outDegree + 1}
        MERGE (i_a)-[:Sends{value: input.val}]->(t))
    '''


def run_tx(tx, event):
    tx.run(query, 
           txid         = event["txid"], 
           output_list  = event["output_list"], 
           block_hash   = event["block_hash"], 
           inDegree     = event["inDegree"], 
           outDegree    = event["outDegree"], 
           outSum       = event["outSum"],
           input_list   = event["input_list"], 
           date         = event["block_date"])
t = tx

In [81]:
#t = t0
with driver.session() as session:
    session.write_transaction(run_tx, t)

## Final TX query

In [27]:
q_final = '''
MERGE (t:Transaction{txid: event.txid})
    SET t += {inDegree: event.inDegree, outDegree: event.outDegree, outSum: event.outSum, date: event.date}

MERGE (b:Block{hash: event.block_hash})
MERGE (t)-[:BELONGS_TO]->(b)

FOREACH (output in event.output_list |
    MERGE (o_a:Address{address: output.addr})
        ON CREATE SET o_a += {inDegree: 1, outDegree: 0}
        ON MATCH  SET o_a += {inDegree: o_a.inDegree + 1}
    MERGE (t)-[r:RECEIVES{output_nr: output.outNr, value: output.val}]->(o_a))

FOREACH (input in event.input_list |
    MERGE (i_a:Address{address: input.addr})
        ON MATCH SET i_a += {outDegree: i_a.outDegree + 1}
    MERGE (i_a)-[:Sends{value: input.val}]->(t))
'''

In [29]:
#### Convert final query by cutting all linebreaks...
omit_characters = ["\n", "     ", "     "]
for i in omit_characters:
    q_final = q_final.replace(i, " ")
q_final

' MERGE (t:Transaction{txid: event.txid}) SET t += {inDegree: event.inDegree, outDegree: event.outDegree, outSum: event.outSum, date: event.date}  MERGE (b:Block{hash: event.block_hash}) MERGE (t)-[:BELONGS_TO]->(b)  FOREACH (output in event.output_list | MERGE (o_a:Address{address: output.addr}) ON CREATE SET o_a += {inDegree: 1, outDegree: 0} ON MATCH  SET o_a += {inDegree: o_a.inDegree + 1} MERGE (t)-[r:RECEIVES{output_nr: output.outNr, value: output.val}]->(o_a))  FOREACH (input in event.input_list | MERGE (i_a:Address{address: input.addr}) ON MATCH SET i_a += {outDegree: i_a.outDegree + 1} MERGE (i_a)-[:Sends{value: input.val}]->(t)) '

## Blocks

#### Creating initial block for backreferencing

In [ ]:
def create_initial_block(tx, initial_hash):
    tx.run('CREATE (b:Block{hash: $initial_hash})', initial_hash=initial_hash)

with driver.session() as session:
    session.write_transaction(create_initial_block, b1['p_block_hash'])

#### Function to create additional blocks

In [ ]:
def create_block(tx, block_hash, block_height, block_timestamp):
    tx.run('MERGE (b:Block{hash: $block_hash, height: $block_height, timestamp: $block_timestamp})',
           block_hash=block_hash, block_height=block_height, block_timestamp=block_timestamp)

In [ ]:
with driver.session() as session:
    session.write_transaction(create_block, b2['block_hash'], b2['block_height'], b2['block_timestamp'])

#### Block Creation and Back Referencing

The function creates the new block, matches the preceding block and creates the relationship between both.
<br>
Issue: breaks if preciding block not creates (not found(?))

In [ ]:
def create_block(tx, block_hash, block_height, block_timestamp, p_block_hash):
    tx.run('MATCH (p:Block) WHERE p.hash = $p_block_hash ' 
           'MERGE (b:Block{hash: $block_hash, height: $block_height, timestamp: $block_timestamp}) '
           'MERGE (p)-[r:Precedes]->(b)',
           block_hash=block_hash, block_height=block_height, block_timestamp=block_timestamp, p_block_hash=p_block_hash)

#### Alternative that recognizes whether the block was already created by the transaction

In [ ]:
query = '''
MATCH (p:Block) WHERE p.hash = $p_block_hash 
MERGE (b:Block {hash: $block_hash}) 
    SET b = {hash: $block_hash, height: $block_height, timestamp: $block_timestamp }
MERGE (p)-[r:Precedes]->(b)
'''

def create_block(tx, block_hash, block_height, block_timestamp, p_block_hash):
    tx.run(query, block_hash=block_hash, block_height=block_height, block_timestamp=block_timestamp, p_block_hash=p_block_hash)

In [ ]:
block = b3
with driver.session() as session:
    session.write_transaction(create_block, 
                              block['block_hash'], 
                              block['block_height'], 
                              block['block_timestamp'],
                              block['p_block_hash'])
    

#### Final Blocks query

In [ ]:
"""
MERGE (p:Block {hash: event.previousblockhash}) 
MERGE (b:Block {hash: event.block_hash}) 
SET b = {hash: event.block_hash, height: event.block_height, blockDate: event.block_date, mediantime: event.block_timestamp } 
MERGE (p)-[r:Precedes]->(b)

"""